In [106]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from collections import Counter


url = "https://date.nager.at/api/v3/publicholidays/2024/BR"
feriados = requests.get(url).json()

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    'latitude': -22.9068,
    'longitude': -43.1729,
    'start_date': '2024-01-01',
    'end_date': '2024-08-01',
    'daily': 'temperature_2m_max,temperature_2m_min,weathercode',
    'temperature_unit': 'celsius',
    'timezone': 'America/Sao_Paulo'
}

clima = requests.get(url, params=params).json()

url = "https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/76b0cb0ef0bfd8a2ec988aa54e30ecd1b483495d/descriptions.json"
descricao = requests.get(url).json()


1. **Quantos feriados há no Brasil em todo o ano de 2024?**

In [107]:
print(f"Há no total {len(feriados)} Feriados")

Há no total 15 Feriados


2. **Qual mês de 2024 tem o maior número de feriados?**

In [108]:
meses = []
for feriado in feriados:
    data = datetime.strptime(feriado['date'], '%Y-%m-%d')
    meses.append(data.strftime("%m"))

contagem = Counter(meses)

mes = contagem.most_common(1)[0][0]
num_feriados = contagem.most_common(1)[0][1]
print(f"O Mês com mais feriados é o Mês {mes} com {num_feriados} Feriados")

O Mês com mais feriados é o Mês 11 com 3 Feriados


3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [109]:
dias_semana = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
feriados_semana = []

for feriado in feriados:
    data = datetime.strptime(feriado['date'], '%Y-%m-%d')
    if data.strftime("%A") in dias_semana:
        feriados_semana.append(feriado)

print(f"{len(feriados_semana)} Feriados caem em dias de semana em 2024")

10 Feriados caem em dias de semana em 2024


4. **Qual foi a temperatura média em cada mês?**  

In [110]:
datas = clima['daily']['time']
temp_max = clima['daily']['temperature_2m_max']
temp_min = clima['daily']['temperature_2m_min']
codigo_clima = clima['daily']['weathercode']

df = pd.DataFrame({
    'data': pd.to_datetime(datas),
    'temp_max': temp_max,
    'temp_min': temp_min,
    'codigo_clima': codigo_clima,
    
})

df['temp_media'] = (df['temp_max'] + df['temp_min']) / 2
df['mes'] = df['data'].dt.month
media_mensal = df.groupby('mes')['temp_media'].mean()
print(media_mensal)


mes
1    27.104839
2    27.722414
3    26.891935
4    25.650000
5    25.583871
6    23.236667
7    21.674194
8    21.800000
Name: temp_media, dtype: float64


5. **Qual foi o tempo predominante em cada mês nesse período?**  

In [111]:
def obter_codigo_clima(codigo_clima):
    return descricao[str(codigo_clima)]['day']['description']

df['descricao_clima'] = df['codigo_clima'].apply(obter_codigo_clima)

descricao_predominante = df.groupby('mes')['descricao_clima'].agg(lambda x: x.mode()[0])
print(descricao_predominante)

mes
1             Rain
2    Light Drizzle
3    Light Drizzle
4           Cloudy
5    Light Drizzle
6           Cloudy
7           Cloudy
8           Cloudy
Name: descricao_clima, dtype: object


6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

In [112]:
df_feriados = pd.DataFrame.from_dict(feriados)
df_feriados.rename(columns={'date': 'data'}, inplace=True)

df['data'] = pd.to_datetime(df['data'])
df_feriados['data'] = pd.to_datetime(df_feriados['data'])

df_filtrado = df[df['data'].isin(df_feriados['data'])].reset_index(drop=True)

df_unido = pd.merge(df_feriados, df_filtrado, left_on='data', right_on='data', how='inner')

for i in df_unido.index:
    feriado = df_unido['localName'][i]
    tempo = df_unido['descricao_clima'][i]
    temp_media = df_unido['temp_media'][i]

    print(f'No Feriado {feriado} o tempo era {tempo} e a temperatura média era {temp_media}')

No Feriado Confraternização Universal o tempo era Light Drizzle e a temperatura média era 24.85
No Feriado Carnaval o tempo era Cloudy e a temperatura média era 30.8
No Feriado Carnaval o tempo era Light Drizzle e a temperatura média era 31.15
No Feriado Sexta-feira Santa o tempo era Light Rain e a temperatura média era 25.700000000000003
No Feriado Domingo de Páscoa o tempo era Drizzle e a temperatura média era 24.7
No Feriado Dia de Tiradentes o tempo era Partly Cloudy e a temperatura média era 23.5
No Feriado Dia do Trabalhador o tempo era Sunny e a temperatura média era 28.35
No Feriado Corpus Christi o tempo era Light Drizzle e a temperatura média era 20.9
No Feriado Revolução Constitucionalista de 1932 o tempo era Drizzle e a temperatura média era 21.95


7. **Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?**

In [113]:
for i in df_unido.index:
    feriado = df_unido['localName'][i]
    tempo = df_unido['descricao_clima'][i]
    temp_media = df_unido['temp_media'][i]

    if temp_media < 20:
        print(f'Feriado {feriado} Não foi aproveitável pois estava Frio')
    elif tempo not in ['Sunny', 'Partly Cloudy']:
        print(f'Feriado {feriado} Não foi aproveitável pois estava Totalmente Nublado ou chuvoso')

Feriado Confraternização Universal Não foi aproveitável pois estava Totalmente Nublado ou chuvoso
Feriado Carnaval Não foi aproveitável pois estava Totalmente Nublado ou chuvoso
Feriado Carnaval Não foi aproveitável pois estava Totalmente Nublado ou chuvoso
Feriado Sexta-feira Santa Não foi aproveitável pois estava Totalmente Nublado ou chuvoso
Feriado Domingo de Páscoa Não foi aproveitável pois estava Totalmente Nublado ou chuvoso
Feriado Corpus Christi Não foi aproveitável pois estava Totalmente Nublado ou chuvoso
Feriado Revolução Constitucionalista de 1932 Não foi aproveitável pois estava Totalmente Nublado ou chuvoso


8. **Qual foi o feriado "mais aproveitável" de 2024?**  

In [ ]:
feriados_mais_proveitoso = ''
melhor_temperatura = 0
for i in df_unido.index:
    feriado = df_unido['localName'][i]
    tempo = df_unido['descricao_clima'][i]
    temp_media = df_unido['temp_media'][i]
    
    if temp_media > 20:
        if tempo in ['Sunny', 'Partly Cloudy']:
            if temp_media > melhor_temperatura:
                feriados_mais_proveitoso = feriado

print(f'O Feriado mais proveitoso foi o {feriados_mais_proveitoso} pois estava')

O Feriado mais proveitoso foi o Dia do Trabalhador
